In [1]:
from typing import Dict, List, Tuple

import pandas as pd
import qlib
from qlib.data import D

from src.dataservice.query2db import get_price

In [2]:
# qlib.init(provider_uri="data/qlib_data", region="cn")

In [3]:
# POOLS: List = D.list_instruments(D.instruments("pool"), as_list=True)
# pct_chg: pd.DataFrame = D.features(POOLS, fields=["CSRank($close)"])

In [4]:
get_price(end_date='2023-06-26',count=2,fields=['close','turnover_rate_f','vol'],fq='hfq')